<img src=https://upload.wikimedia.org/wikipedia/commons/6/68/Logo_universidad_icesi.svg width=300>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/sebastianb92/nlp-labs/blob/main/Session2/1-Word-Embeddings.ipynb)


# Maestría en Inteligencia Artificial  
## Procesamiento de Lenguaje natural
### Sesión 2 - Práctica

---


**Integrantes:**  
- Johan Sebastian Bonilla  
- Edwin Gómez  

# Introducción

# Análisis Semántico de Eventos de Seguridad con Word2Vec

En este notebook utilizamos Word2Vec (gensim) para representar palabras en un espacio vectorial
y analizar descripciones generadas por un sistema de detección de armas.

Objetivo:
Usar similitud coseno en el espacio vectorial para estimar nivel de riesgo.

# Configurar entorno

En esta sección se configuran las librerías y dependencias necesarias para el análisis de datos y procesamiento de lenguaje natural. Esto garantiza que el entorno esté listo para cargar, limpiar y analizar las conversaciones del chat político.

In [1]:
import sys
import warnings

warnings.filterwarnings('ignore')

# Detectar si estamos en Google Colab
IN_COLAB = 'google.colab' in sys.modules

In [2]:
if IN_COLAB:
    !wget https://raw.githubusercontent.com/sebastianb92/nlp-labs/refs/heads/main/requirements.txt && pip install -r requirements.txt
    !python -m spacy download es_core_news_sm
else:
    !pip3 install -r ../requirements.txt

--2026-02-26 23:33:47--  https://raw.githubusercontent.com/sebastianb92/nlp-labs/refs/heads/main/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274 [text/plain]
Saving to: ‘requirements.txt’

requirements.txt    100%[===================>]     274  --.-KB/s    in 0s      

2026-02-26 23:33:47 (5.41 MB/s) - ‘requirements.txt’ saved [274/274]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.9/12.9 MB 111.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('es_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab n

In [4]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 76.9 MB/s eta 0:00:00


In [5]:
import gensim
from gensim.models import Word2Vec
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [6]:
descripciones = [
    "persona caminando en estacion de bus",
    "grupo de personas esperando transporte publico",
    "persona armada con cuchillo en servicio publico",
    "hombre con pistola dentro de autobus",
    "ninos jugando en parque",
    "sujeto con machete en calle principal"
]

In [7]:
corpus = [frase.split() for frase in descripciones]

In [8]:
model = Word2Vec(
    sentences=corpus,
    vector_size=100,
    window=5,
    min_count=1,
    workers=4
)

In [9]:
model.wv.similarity("cuchillo", "pistola")

np.float32(0.24664757)

In [10]:
model.wv.similarity("cuchillo", "parque")

np.float32(-0.1604416)

In [11]:
model.wv.most_similar("cuchillo")

[('pistola', 0.24664753675460815),
 ('armada', 0.11960580945014954),
 ('machete', 0.11939920485019684),
 ('dentro', 0.11676604300737381),
 ('jugando', 0.09616676717996597),
 ('estacion', 0.08788354694843292),
 ('con', 0.08547668159008026),
 ('principal', 0.07164211571216583),
 ('ninos', 0.059755709022283554),
 ('publico', 0.04116104170680046)]

In [12]:
def embedding_frase(frase):
    palabras = frase.split()
    vectores = []

    for palabra in palabras:
        if palabra in model.wv:
            vectores.append(model.wv[palabra])

    return np.mean(vectores, axis=0)

In [13]:
concepto_riesgo = embedding_frase(
    "arma cuchillo pistola machete violencia amenaza ataque"
)

In [14]:
for frase in descripciones:
    emb = embedding_frase(frase)
    sim = cosine_similarity([emb], [concepto_riesgo])[0][0]
    print(f"{frase} → Riesgo: {sim:.3f}")

persona caminando en estacion de bus → Riesgo: -0.006
grupo de personas esperando transporte publico → Riesgo: 0.016
persona armada con cuchillo en servicio publico → Riesgo: 0.342
hombre con pistola dentro de autobus → Riesgo: 0.279
ninos jugando en parque → Riesgo: -0.023
sujeto con machete en calle principal → Riesgo: 0.179


In [15]:
def clasificar_riesgo(score):
    if score > 0.5:
        return "ALTO"
    elif score > 0.25:
        return "MEDIO"
    else:
        return "BAJO"

for frase in descripciones:
    emb = embedding_frase(frase)
    sim = cosine_similarity([emb], [concepto_riesgo])[0][0]
    nivel = clasificar_riesgo(sim)
    print(f"{frase} → {nivel}")

persona caminando en estacion de bus → BAJO
grupo de personas esperando transporte publico → BAJO
persona armada con cuchillo en servicio publico → MEDIO
hombre con pistola dentro de autobus → MEDIO
ninos jugando en parque → BAJO
sujeto con machete en calle principal → BAJO


In [16]:
model.wv.most_similar(
    positive=["cuchillo", "persona"],
    negative=["parque"]
)

[('dentro', 0.25052058696746826),
 ('principal', 0.1661691516637802),
 ('pistola', 0.10847946256399155),
 ('de', 0.1077161580324173),
 ('con', 0.09780752658843994),
 ('machete', 0.09694948047399521),
 ('publico', 0.08733217418193817),
 ('armada', 0.06975025683641434),
 ('autobus', 0.06755538284778595),
 ('ninos', 0.06742879003286362)]

In [17]:
model.wv.most_similar(
    positive=["pistola", "autobus"]
)

[('bus', 0.1930246502161026),
 ('cuchillo', 0.12120603024959564),
 ('publico', 0.11435778439044952),
 ('transporte', 0.10839338600635529),
 ('con', 0.10621056705713272),
 ('personas', 0.06334301829338074),
 ('grupo', 0.05893673002719879),
 ('jugando', 0.055236801505088806),
 ('caminando', 0.046140559017658234),
 ('machete', 0.04375447705388069)]

Limitación:
Word2Vec requiere grandes corpus para aprender representaciones robustas.
Este ejemplo es demostrativo.

## Resultados

Aunque la traducción mejora los resultados, persisten algunas limitaciones:

- **Pérdida de matices idiomáticos**: expresiones coloquiales colombianas pueden no traducirse bien.
- **Sarcasmo e ironía**: difíciles de preservar en traducción automática y aún con una buena traducción el sarcasmo es difícilmente interpretado por este modelo simple.
- **Contexto cultural**: referencias locales (RTVC, Petro) pueden perder significado.
- **Latencia**: la traducción añade tiempo de procesamiento.

### Hallazgos del Análisis

Los resultados con traducción revelan:
- **Mayor variabilidad** en los scores compound (ya no todo es neutral).
- **Detección de polarización**: se identifican mensajes claramente positivos y negativos, aunque no precisos debido al alto nivel de sarcasmo empleado en el chat.

### Conclusión

El enfoque de **traducción + VADER** es práctico en un entorno académico de análisis de sentimientos en español, pero pierde algunos matices culturales y no se recomienda esta aproximación en un entorno real. Se captura de mejor manera la polaridad general de los mensajes comparado con el tratamiento de los mensajes originales en español y permite identificar patrones de comunicación entre los participantes del chat político analizado.